# Exceptions

In backend development, there are situations where you need to return messages to the user indicating errors during code execution. There are dedicated objects designed for this purpose: `HTTPException`, `RequestValidationError`, `StarletteHTTPException`, and so on. You can even create your custom exceptions. The key feature of these objects is that you must *raise* them, not return them.

Find out more in [handling errors](https://fastapi.tiangolo.com/tutorial/handling-errors/#raise-an-httpexception-in-your-code) official fastAPI documentation.

In [15]:
!docker run --rm -itd\
    --name test_container\
    -v ./exceptions_files/app.py:/app.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload app:app

be0983c1edd190b76090884f7a4befef69fe6201528140fc0049cf9706c58f16


**Note** don't forget to stop container after all your tests.

In [29]:
!docker stop test_container

Error response from daemon: No such container: test_container


## Custom

You can create a custom exception by defining a class that inherits from `Exception`. Then, use the `app.exception_handler` decorator to associate this exception class with a specific function.

---

The following example demonstrates how to handle a custom exception. The class `MyException` inherits from `Exception`, and instances of this class can be raised as exceptions. The `unicorn_exception_handler` function is decorated with `app.exception_handler(MyException)`, ensuring that it is invoked whenever a `MyException` is raised.

In [26]:
%%writefile exceptions_files/app.py
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse


class MyException(Exception):
    def __init__(self, message):
        self.message = message

app = FastAPI()

@app.exception_handler(MyException)
def unicorn_exception_handler(request: Request, exc: MyException):
    return JSONResponse(
        status_code=418,
        content={"message": f"Oops! {exc.message} did something."}
    )

@app.get("/")
def divide():
    raise MyException("My god")

Overwriting exceptions_files/app.py


Now, let's test the newly created API with a request.

In [27]:
!curl localhost:8000

{"message":"Oops! My god did something."}

We received a typical FastAPI error message, but it was customized according to the behavior I specified.